In [ ]:
CUDA_VISIBLE_DEVICES=0,1 torchrun --nproc_per_node 2 \
-m FlagEmbedding.baai_general_embedding.finetune.run \
--output_dir /workspace/data/private/zhuxiaohai/models/bge_emb_finetune_from_db \
--model_name_or_path /data/dataset/huggingface/hub/bge-large-zh-v1.5 \
--train_data /data/dataset/kefu/bge_finetune_emb_from_db2.jsonl \
--learning_rate 1e-5 \
--fp16 \
--num_train_epochs 5 \
--per_device_train_batch_size 4 \
--negatives_cross_device False \
--dataloader_drop_last False \
--normlized True \
--temperature 0.02 \
--query_max_len 96 \
--passage_max_len 256 \
--train_group_size 6 \
--logging_steps 10 \
--use_inbatch_neg True \
--save_steps 100 \
--query_instruction_for_retrieval "为这个句子生成表示以用于检索相关文章：" \
--gradient_accumulation_steps 8

In [1]:
import numpy as np 
import os
import pandas as pd 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 使用GPU 0

In [2]:
dd = pd.read_csv("/data/dataset/kefu/emb.csv")
dd.columns = ['question', 'emb']

In [3]:
from FlagEmbedding import FlagModel

In [8]:
model = FlagModel('/workspace/data/private/zhuxiaohai/models/bge_finetune_emb', use_fp16=False) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [9]:
q_embeddings = model.encode(dd.question.tolist())

In [11]:
from sentence_transformers import SentenceTransformer

model2 = SentenceTransformer('/workspace/data/private/zhuxiaohai/models/bge_finetune_emb')

q_embeddings2 = model2.encode(dd.question.tolist(), normalize_embeddings=True, batch_size=32)

In [12]:
q_embeddings2.shape

(38, 1024)

In [13]:
for i in range(dd.shape[0]):
    np.testing.assert_array_equal(q_embeddings2[i], q_embeddings2[i])